# Tarea Apache Spark
>Importa las librerías necesarias dónde sea necesario


In [ ]:
import pandas as pd 
from pyspark.sql import SparkSession

### SparkSession
>Crea un SparkSession para comenzar la tarea

### Crear un DataFrame
>Lee el csv datosTarea.csv, mételo a un DF y muéstralo.

### Fistro de datos
>Consigue todas las empresas que empiecen con 'M' y tengan entre 4000 y 7000 empleados. Sólo muestra los nombres y el número de empleados.

>Consigue todos los países que no inicien con las letras 'b', 's' y 'm', pero que tampoco tengan un netword mayor a 500000. Muestra el nombre de la compañía, el país y el networth.

### Funciones
Crea una función con @pandas_udf que que le reste a los profits la media en cada renglón. Crea una nueva columna que muestre los resultados.

### Grouping data
>Agrupa por industry y muestra cuáles son las empresas con el profit más alto. Muestra los primeros tres.

>Agrupa por industry y calcula el promedio de empleados que tienen

### SQL
>Usando Spark SQL, obtén cuántas empresas se fundaron despúes del 2000.

### ML Regresión Lineal
>Con número de empleados, networth y stock price, obtén una predicción del profit a través de una regresión lineal.